In [1]:
import pandas as pd

In [2]:
import sys

In [3]:
import numpy as np

In [4]:
import tqdm.notebook as tqdm

In [5]:
import math

In [6]:
import tensorflow as tf

In [70]:
## We start by purifying our databases

In [22]:
file = pd.read_json("uspto-templates.json")

In [23]:
file2 = pd.read_json("uspto-reactions.json")

In [50]:
l1 = file['reaction_id'].to_list()

In [54]:
l2 = file2['_id'].to_list()

In [71]:
ids_not_in_templates = list(set(l2)-set(l1))

In [94]:
nan = file['reaction_smarts'].to_numpy()[8]

In [88]:
data = file['reaction_smarts'].to_numpy()

In [119]:
rows_with_nan = []
pb = tqdm.tqdm(total=1800000)
for index, row in file.iterrows():
    pb.update(1)
    is_nan_series = row.isnull()
    if is_nan_series.any():
        rows_with_nan.append(index)

  0%|          | 0/1800000 [00:00<?, ?it/s]

In [150]:
reac_id_nan = []
for val in rows_with_nan:
    reac_id_nan.append(file.iloc[val,0])

In [151]:
to_delete = []
for val in reac_id_nan:
    to_delete.append(df.index[df["_id"]==val].to_list())

In [153]:
to_delete = [i for f in to_delete for i in f]

In [157]:
df.drop(to_delete, axis=0, inplace=True)

In [159]:
file.drop(rows_with_nan, axis=0, inplace=True)

In [162]:
file.to_csv("Purified_templates.csv", index = False)

In [163]:
df.to_csv("Purified_reactions.csv", index = False)

In [125]:
df = file2

In [129]:
x = []
for val in ids_not_in_templates:
    x.append(df.index[df["_id"] == val].to_list())

In [132]:
ids_to_delete = [i for f in x for i in f]

In [135]:
df.drop(ids_to_delete, axis=0,inplace=True)

In [144]:
df.drop(rows_with_nan, axis = 0, inplace=True)

KeyError: '[108905, 280174, 869040, 869041, 869042, 1661986] not found in axis'

In [7]:
from tensorflow.keras.utils import to_categorical

In [7]:
n = data_templates.shape[0]

In [34]:
data = file.drop_duplicates(subset="reaction_smarts")

In [45]:
values = data['reaction_smarts'].to_list()
my_map = dict(list(enumerate(values)))

In [46]:
inv_map = {v: k for k, v in my_map.items()}

In [29]:
def fp_from_fp_db(fp_db):
    res = np.zeros(2048)
    list_idx = fp_db.split(" ")
    for el in list_idx[:-1]:
        lfp = el.split("-")
        n = len(lfp)
        if n == 1:
            res[int(lfp[0])] = 1
        elif n==2:
            res[int(lfp[0])] = int(lfp[-1])
        else:
            res[int(lfp[0])] = -int(lfp[-1])
    return res

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, n_channels=1,
                 n_classes=10, shuffle=True, _filename_fp="", _filename_templates = "", _filename_reactions=""):
        'Initialization'
        
        self.batch_size = batch_size
        
        self.list_IDs = list_IDs
        
        self.n_channels = n_channels
        
        self.n_classes = n_classes
        
        self.shuffle = shuffle
        
        self.filename_fp = _filename_fp
        self.filename_templates = _filename_templates
        self.filename_reactions = _filename_reactions
        
        self.data_fp = pd.read_csv(_filename_fp)
        self.data_templates = pd.read_csv(_filename_templates)
        self.data_reactions = pd.read_csv(_filename_reactions)
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, 2048))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = fp_from_fp_db(self.data_fp.iloc[i,1]-self.data_fp.iloc[i,2])

            # Store class
            y[i] = self.data_reactions.iloc[i,0]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
    
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
import numpy as np

from keras.models import Sequential
from my_classes import DataGenerator

# Parameters
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

# Datasets
partition = # IDs
labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential()
[...] # Architecture
model.compile()

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)

In [13]:
(sys.getsizeof(np.zeros(300000))/1000*300000)/1000

720033.6

In [ ]:

model = Sequential()
model.add(Dense(512,input_shape=(2048,),activation="elu"))
model.add(Dropout(0.4))
model.add(Dense(n, activation="softmax"))
